<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=9aa2e0dd7e7912d81110bfd29d58a1c88ed299da66d77ab809404ceadf30d180
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-17 12:23:56
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.41 C
-------------------
Today PnL: 41.69 K (0.3%)
Current PnL: -20.66 L (-13.76%)
CY Booked + Current PnL: -8.76 L (-5.84%)
-------------------
Total profit:  1.99 L
Total loss:  -22.65 L
-------------------
Total Booked + Current PnL: 18.22 L (14.8%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.45%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.82 L (62.37%)
Deployed:  1.23 C
Current:  1.41 C
CAGR/XIRR %: 7.98%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-19.78,57.0,X-LC,3.87,214474.0,9974.0,13319.0,-0.25,4.88,6.21,11.39,10.0,0.75,1.53,14.27,X40N,ATH,PHARMA
83,VOLTAS,1530.00,2.56,62.0,X-MC,1.37,215175.0,23433.0,14331.0,1.34,12.22,6.66,19.69,99.0,1.64,1.54,19.85,XY25,NTT,AC
76,TTKPRESTIG,770.00,97.65,52.0,M-SC,1.62,85163.0,-15614.0,15704.0,-0.61,-15.49,18.44,0.09,245.0,-0.99,0.61,11.89,OX40N,NTT,DURABLES
56,RELIANCE,1533.00,-11.41,64.0,X-LC,2.85,221645.0,11279.0,17510.0,1.61,5.36,7.90,13.68,37.0,0.64,1.59,22.77,XY25,NTT,REFINERIES
33,ICICIGI,2260.25,-13.61,66.0,X-MC,2.35,148696.0,12563.0,18557.0,-0.03,9.23,12.48,22.86,91.0,0.68,1.06,23.01,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-39.78,67.0,M-SC,4.23,101768.0,10270.0,68480.0,9.62,11.22,67.29,86.07,223.0,0.15,0.73,47.78,XR,NTT,DURABLES
5,ASIANPAINT,3465.66,-11.88,66.0,X-LC,5.31,222332.0,-29436.0,82641.0,4.85,-11.69,37.17,21.13,27.0,-0.36,1.59,19.59,X40,ATH,PAINTS
44,KANSAINER,340.00,-66.69,63.0,H-SC,1.93,229419.0,-40248.0,76580.0,3.98,-14.93,33.38,13.47,138.0,-0.53,1.64,16.65,XY24,NTT,PAINTS
4,ANGELONE,3033.00,17.86,72.0,X-SC,8.74,208887.0,17881.0,39814.0,2.61,9.36,19.06,30.21,157.0,0.45,1.49,32.20,X40N,NTT,FINANCE
25,GREENPANEL,537.00,233.33,57.0,M-SC,4.14,148840.0,-34238.0,113223.0,2.54,-18.70,76.07,43.14,230.0,-0.30,1.07,40.00,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-14.70,43.0,M-LC,5.71,149637.0,-1308.0,110761.0,-4.91,-0.87,74.02,72.51,53.0,-0.01,1.07,5.10,XR,NTT,IT
15,CAMPUS,393.0,-23.71,54.0,M-SC,2.78,158922.0,-8853.0,64697.0,-2.03,-5.28,40.71,33.28,210.0,-0.14,1.14,23.18,XY24,NTT,FOOTWEAR
7,ATULAUTO,844.0,3635.14,50.0,M-SC,3.75,167654.0,-29478.0,119303.0,-2.02,-14.95,71.16,45.57,236.0,-0.25,1.20,18.86,XY24,NTT,AUTO
39,INFY,2275.0,-21.03,41.0,X-LC,7.48,307828.0,-5060.0,176755.0,-1.79,-1.62,57.42,54.87,3.0,-0.03,2.20,4.92,X40,BTT,IT
63,SHALBY,327.0,1164.75,52.0,M-SC,23.33,172072.0,-9071.0,54547.0,-1.76,-5.01,31.70,25.10,235.0,-0.17,1.23,37.90,XY24,NTT,HEALTHCARE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-187.50,70.0,M-SC,18.71,150435.0,1215.0,142958.0,0.16,0.81,95.03,96.62,208.0,0.01,1.08,62.80,XY25,NTT,FINANCE
36,INDIAMART,4810.62,-55.39,40.0,H-SC,7.91,124550.0,1214.0,130416.0,-0.26,0.98,104.71,106.72,119.0,0.01,0.89,24.59,AR,ATH,MISC
54,RECLTD,446.00,45.36,46.0,M-LC,6.24,204738.0,1568.0,40559.0,-1.39,0.77,19.81,20.74,55.0,0.04,1.47,6.36,XY25,NTT,FINANCE
85,WIPRO,420.00,-14.70,43.0,M-LC,5.71,149637.0,-1308.0,110761.0,-4.91,-0.87,74.02,72.51,53.0,-0.01,1.07,5.10,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,107.70,60.0,H-SC,12.32,131382.0,-11097.0,30612.0,-0.67,-7.79,23.30,13.70,163.0,-0.36,0.94,55.07,OX40N,NTT,BANKS
44,KANSAINER,340.0,-66.69,63.0,H-SC,1.93,229419.0,-40248.0,76580.0,3.98,-14.93,33.38,13.47,138.0,-0.53,1.64,16.65,XY24,NTT,PAINTS
76,TTKPRESTIG,770.0,97.65,52.0,M-SC,1.62,85163.0,-15614.0,15704.0,-0.61,-15.49,18.44,0.09,245.0,-0.99,0.61,11.89,OX40N,NTT,DURABLES
17,CERA,9475.0,-23.87,40.0,H-SC,2.35,138759.0,-37144.0,79162.0,-0.10,-21.12,57.05,23.89,149.0,-0.47,0.99,19.84,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.72,44.0,H-MC,7.35,101878.0,-29957.0,72181.0,-0.77,-22.72,70.85,32.03,98.0,-0.42,0.73,14.11,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,46.0,M-LC,6.24,204738.0,1568.0,40559.0,-1.39,0.77,19.81,20.74,55.0,0.04,1.47,6.36,XY25,NTT,FINANCE
59,SAIL,228.00,44.35,51.0,M-MC,12.17,227252.0,2290.0,165371.0,0.45,1.02,72.77,74.53,192.0,0.01,1.63,33.60,XY24,BTT,STEEL
31,HINDZINC,730.22,34.91,64.0,M-LC,10.03,219588.0,14512.0,98068.0,-0.37,7.08,44.66,54.89,52.0,0.15,1.57,32.88,X5K,ATH,METALS
78,UNIONBANK,163.00,-13.18,53.0,M-LC,8.79,157133.0,16293.0,28205.0,-0.44,11.57,17.95,31.59,66.0,0.58,1.12,41.12,XY24,NTT,BANKS
11,BANKINDIA,190.00,-25.47,60.0,H-MC,12.26,192788.0,12980.0,99440.0,-0.06,7.22,51.58,62.52,88.0,0.13,1.38,43.77,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,34.91,64.0,M-LC,10.03,219588.0,14512.0,98068.0,-0.37,7.08,44.66,54.89,52.0,0.15,1.57,32.88,X5K,ATH,METALS
84,WHIRLPOOL,2270.00,-39.78,67.0,M-SC,4.23,101768.0,10270.0,68480.0,9.62,11.22,67.29,86.07,223.0,0.15,0.73,47.78,XR,NTT,DURABLES
11,BANKINDIA,190.00,-25.47,60.0,H-MC,12.26,192788.0,12980.0,99440.0,-0.06,7.22,51.58,62.52,88.0,0.13,1.38,43.77,XR,NTT,BANKS
36,INDIAMART,4810.62,-55.39,40.0,H-SC,7.91,124550.0,1214.0,130416.0,-0.26,0.98,104.71,106.72,119.0,0.01,0.89,24.59,AR,ATH,MISC
85,WIPRO,420.00,-14.70,43.0,M-LC,5.71,149637.0,-1308.0,110761.0,-4.91,-0.87,74.02,72.51,53.0,-0.01,1.07,5.10,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,7.44,144329.0,-132194.0,242271.0,0.20,-47.81,167.86,39.81,54.0,-0.55,1.03,1.73,XY24,NTT,AUTO
51,QUESS,986.00,-48.21,33.0,X-SC,38.66,51695.0,-13311.0,164240.0,-0.96,-20.48,317.71,232.18,198.0,-0.08,0.37,0.81,XY24,NTT,MISC
28,HAVELLS,2069.16,-6.75,41.0,X-MC,2.12,231301.0,-16430.0,91480.0,0.47,-6.63,39.55,30.30,92.0,-0.18,1.66,5.78,X40,ATH,ELECTRICAL
39,INFY,2275.00,-21.03,41.0,X-LC,7.48,307828.0,-5060.0,176755.0,-1.79,-1.62,57.42,54.87,3.0,-0.03,2.20,4.92,X40,BTT,IT
9,BAJAJHFL,181.50,-14.90,42.0,X-MC,6.04,179252.0,-21254.0,118414.0,-0.67,-10.60,66.06,48.45,90.0,-0.18,1.28,3.37,X40N,ATH,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-2.40,47.0,H-LC,0.43,156880.0,-29215.0,76699.0,0.60,-15.70,48.89,25.51,15.0,-0.38,1.12,15.61,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,2.56,62.0,X-MC,1.37,215175.0,23433.0,14331.0,1.34,12.22,6.66,19.69,99.0,1.64,1.54,19.85,XY25,NTT,AC
28,HAVELLS,2069.16,-6.75,41.0,X-MC,2.12,231301.0,-16430.0,91480.0,0.47,-6.63,39.55,30.30,92.0,-0.18,1.66,5.78,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-13.61,66.0,X-MC,2.35,148696.0,12563.0,18557.0,-0.03,9.23,12.48,22.86,91.0,0.68,1.06,23.01,X40,ATH,INSURANCE
41,ITC,452.00,-37.50,61.0,X-LC,2.62,202153.0,2015.0,18881.0,2.04,1.01,9.34,10.44,4.0,0.11,1.45,6.63,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,33.0,X-SC,38.66,51695.0,-13311.0,164240.0,-0.96,-20.48,317.71,232.18,198.0,-0.08,0.37,0.81,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,7.44,144329.0,-132194.0,242271.0,0.20,-47.81,167.86,39.81,54.0,-0.55,1.03,1.73,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-14.90,42.0,X-MC,6.04,179252.0,-21254.0,118414.0,-0.67,-10.60,66.06,48.45,90.0,-0.18,1.28,3.37,X40N,ATH,FINANCE
73,TCS,4389.96,-29.65,47.0,X-LC,12.62,280082.0,-65874.0,132563.0,0.30,-19.04,47.33,19.28,1.0,-0.50,2.00,3.54,X40,ATH,IT
2,ACC,3906.00,-37.64,50.0,X-MC,3.25,185390.0,-52561.0,205208.0,-0.33,-22.09,110.69,64.15,174.0,-0.26,1.33,3.99,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,33.0,X-SC,38.66,51695.0,-13311.0,164240.0,-0.96,-20.48,317.71,232.18,198.0,-0.08,0.37,0.81,XY24,NTT,MISC
55,RELAXO,1176.00,-39.68,43.0,X-SC,4.41,82212.0,-62948.0,138873.0,-0.88,-43.36,168.92,52.31,136.0,-0.45,0.59,10.05,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-35.72,44.0,X-SC,4.30,96031.0,-32639.0,77939.0,0.81,-25.37,81.16,35.20,219.0,-0.42,0.69,10.22,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-4.94,51.0,X-SC,0.83,107999.0,5995.0,37929.0,0.69,5.88,35.12,43.06,122.0,0.16,0.77,24.00,X40N,ATH,MISC
32,HONAUT,58357.33,-23.00,46.0,X-SC,2.52,108570.0,-19368.0,66499.0,0.28,-15.14,61.25,36.84,143.0,-0.29,0.78,11.15,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-29.65,47.0,X-LC,12.62,280082.0,-65874.0,132563.0,0.30,-19.04,47.33,19.28,1.0,-0.50,2.00,3.54,X40,ATH,IT
39,INFY,2275.00,-21.03,41.0,X-LC,7.48,307828.0,-5060.0,176755.0,-1.79,-1.62,57.42,54.87,3.0,-0.03,2.20,4.92,X40,BTT,IT
41,ITC,452.00,-37.50,61.0,X-LC,2.62,202153.0,2015.0,18881.0,2.04,1.01,9.34,10.44,4.0,0.11,1.45,6.63,X40,NTT,FMCG
82,VBL,671.64,-17.48,57.0,X-LC,10.48,297276.0,-18566.0,131218.0,1.01,-5.88,44.14,35.67,5.0,-0.14,2.13,7.16,X40N,ATH,FMCG
1,ABB,7934.00,-40.16,51.0,H-LC,7.08,250128.0,-11491.0,130692.0,-0.02,-4.39,52.25,45.57,7.0,-0.09,1.79,6.43,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6800.00,59.0,L-SC,6.26,75685.0,-18125.0,94742.0,0.71,-19.32,125.18,81.67,269.0,-0.19,0.54,46.57,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,29.13,80640.0,-32909.0,72963.0,0.60,-28.98,90.48,35.27,268.0,-0.45,0.58,97.37,XR,NTT,HOTELS
48,MASFIN,398.61,-20.96,41.0,H-SC,9.47,89850.0,-8130.0,29731.0,-1.11,-8.30,33.09,22.05,152.0,-0.27,0.64,30.60,XR,ATH,FINANCE
84,WHIRLPOOL,2270.00,-39.78,67.0,M-SC,4.23,101768.0,10270.0,68480.0,9.62,11.22,67.29,86.07,223.0,0.15,0.73,47.78,XR,NTT,DURABLES
77,UJJIVANSFB,60.00,107.70,60.0,H-SC,12.32,131382.0,-11097.0,30612.0,-0.67,-7.79,23.30,13.70,163.0,-0.36,0.94,55.07,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-187.50,70.0,M-SC,18.71,150435.0,1215.0,142958.0,0.16,0.81,95.03,96.62,208.0,0.01,1.08,62.80,XY25,NTT,FINANCE
4,ANGELONE,3033.00,17.86,72.0,X-SC,8.74,208887.0,17881.0,39814.0,2.61,9.36,19.06,30.21,157.0,0.45,1.49,32.20,X40N,NTT,FINANCE
67,STARHEALTH,761.00,21.75,70.0,H-SC,9.17,270994.0,-10129.0,143004.0,1.57,-3.60,52.77,47.26,144.0,-0.07,1.94,45.19,XY24,NTT,INSURANCE
31,HINDZINC,730.22,34.91,64.0,M-LC,10.03,219588.0,14512.0,98068.0,-0.37,7.08,44.66,54.89,52.0,0.15,1.57,32.88,X5K,ATH,METALS
63,SHALBY,327.00,1164.75,52.0,M-SC,23.33,172072.0,-9071.0,54547.0,-1.76,-5.01,31.70,25.10,235.0,-0.17,1.23,37.90,XY24,NTT,HEALTHCARE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.62
1,25,44.26
2,50,75.93


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.34
LC    33.81
MC    21.85
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.60
X40      14.80
XY25     12.39
XR       10.89
X40N     10.24
AR        8.11
OX40N     7.62
X200      1.82
X5K       1.57
SR        1.12
MH        0.84
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.42
X-LC    22.82
X-MC    16.12
M-SC    12.77
M-LC     5.23
H-LC     4.73
X-SC     4.69
H-MC     3.79
M-MC     1.63
L-SC     1.46
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.33,-2.54,36.33
IT,12.76,-17.93,81.58
FINANCE,12.18,-11.12,61.49
MISC,6.86,-19.88,86.30
BANKS,6.34,-12.46,71.62
PAINTS,5.72,-15.35,32.54
ELECTRICAL,5.35,-10.24,48.92
INSURANCE,3.96,0.15,37.18
AUTO,2.81,-46.04,115.85


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3298285.0,22
XR,1387249.0,14
AR,1196399.0,9
X40,768687.0,10
X40N,595431.0,8
OX40N,582181.0,10
XY25,453858.0,8
SR,264350.0,2
X5K,98068.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3445954.0,25
M-SC,1258872.0,15
X-MC,1072707.0,11
X-LC,1049693.0,13
X-SC,525294.0,6
H-MC,350313.0,3
H-LC,278899.0,3
M-LC,277593.0,4
L-SC,257703.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1181292.0      6
           XR         831858.0      7
           AR         827383.0      5
M-SC       XY24       690861.0      6
X-MC       XY24       580874.0      4
X-LC       X40        477144.0      5
           XY24       308750.0      2
X-SC       X40N       283115.0      4
H-SC       OX40N      274628.0      4
           SR         264350.0      2
X-MC       X40        213604.0      4
H-LC       AR         207391.0      2
X-LC       X40N       193902.0      3
H-MC       XY24       178692.0      1
L-SC       XR         167705.0      2
M-MC       XY24       165371.0      1
X-SC       XY24       164240.0      1
M-SC       AR         161625.0      2
X-MC       XY25       159815.0      2
M-SC       OX40N      145374.0      4
           XY25       142958.0      1
X-MC       X40N       118414.0      1
M-SC       XR         118054.0      2
M-LC       XR         110761.0      1
H-MC       XR          99440.0      1
M-LC       X5K         98068.0      1
L-SC       OX40N       89998.0      1
X-SC       X40         77939.0      1
H-MC       OX40N       72181.0      1
H-LC       X200        71508.0      1
X-LC       XY25        69897.0      3
H-SC       MH          66443.0      1
L-MC       XR          59431.0      1
L-LC       XY25        40629.0      1
M-LC       XY25        40559.0      1
           XY24        28205.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
